In [ ]:
# Elmo

import tensorflow as tf
import tensorflow_hub as hub

url = "https://tfhub.dev/google/elmo/3"
embed = hub.load(url)

from keras.layers import Input, Lambda, Dense,Layer
from keras.models import Model
from keras import backend as K

# Initialize session
sess = tf.compat.v1.Session()
K.set_session(sess)

In [ ]:
trainx=X_train['tweet_initial_nontoken'].tolist()
trainx = [' '.join(t.split()[0:150]) for t in trainx]
trainx = np.array(trainx, dtype=object)[:, np.newaxis]

testx = X_test['tweet_initial_nontoken'].tolist()
testx = [' '.join(t.split()[0:150]) for t in testx]
testx = np.array(testx, dtype=object)[:, np.newaxis]

In [ ]:
# Create a custom layer that allows us to update weights (lambda layers do not have trainable parameters!)

class ElmoEmbeddingLayer(Layer):
    def __init__(self, **kwargs):
        self.dimensions = 1024
        self.trainable=True
        super(ElmoEmbeddingLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        self.elmo = hub.Module('https://tfhub.dev/google/elmo/2', trainable=self.trainable,
                               name="{}_module".format(self.name))

      #  self.trainable_weights += K.tf.trainable_variables(scope="^{}_module/.*".format(self.name))
      #  super(ElmoEmbeddingLayer, self).build(input_shape)

    def call(self, x, mask=None):
        result = self.elmo(K.squeeze(K.cast(x, tf.string), axis=1),
                      as_dict=True,
                      signature='default',
                      )['default']
        return result

    def compute_mask(self, inputs, mask=None):
        return K.not_equal(inputs, '--PAD--')

    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.dimensions)

In [ ]:

# Function to build model
"""
  model.add(Conv1D(128, 3, activation='relu'))
  model.add(MaxPooling1D(pool_size=4))
  model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
  model.add(layers.Dense(100, activation='relu'))
  model.add(Dense(1, activation='sigmoid'))"""

def build_model(): 
  input_text = layers.Input(shape=(1,), dtype="string")
  embedding = ElmoEmbeddingLayer()(input_text)
  #(embedding.shape)
  #conv#1 = Conv1D(128, kernel_size=3, activation='relu')(K.expand_dims(embedding,0))
  #pool1 = MaxPooling1D(pool_size=4)(conv1)
  #conv2 = Conv1D(16, kernel_size=1, activation='relu')(pool1)
  #pol2 = MaxPooling1D(pool_size=2)(conv2)
  #flat = Flatten()(K.get_shape(pool1,-1,1))
  #print("a")
  dense = layers.Dense(128, activation='relu')(embedding)

  pred = layers.Dense(1, activation='sigmoid')(dense)

  model = Model(inputs=[input_text], outputs=pred)

  #model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
  model.summary()
  
  return model

In [ ]:
model_elmo=build_model()
early_stopping = [EarlyStopping(monitor='val_acc',min_delta=0,restore_best_weights=True, patience=3,verbose=1, mode='auto')]
model_elmo.compile(optimizer=keras.optimizers.Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, amsgrad=False),loss='binary_crossentropy',metrics=['acc',f1_m,precision_m, recall_m])

model_elmo.fit(trainx, y_train,
            batch_size=16,
            epochs=5,
            callbacks=early_stopping,
            validation_split=0.1)

In [ ]:
loss, accuracy, f1_score, precision, recall = model_elmo.evaluate(testx, y_test, verbose=1)
print("elmo Test Loss: {:.4f}".format(loss))
print("elmo Test Accuracy: {:.4f}".format(accuracy))
print("elmo Test f1 score: {:.4f}".format(f1_score))
print("elmo  Test Precision: {:.4f}".format(precision))
print("elmo BiRNN Test Recall: {:.4f}".format(recall))
model_elmo.save('ElmoModel.h5')

probs = model_elmo.predict(testx, verbose=1)
print('lenght of probs : ' ,len(probs))
  #predicted_classes = probs.argmax(axis=-1)
predicted_classes=[0 if i < 0.5 else 1 for i in probs]
  #predicted_classes = attention_bilstm_model.predict_classes(X_test_initial, verbose=1)
  #print(predicted_classes)

print(classification_report(y_test, predicted_classes))

In [ ]:
from sklearn.metrics import confusion_matrix

confusion_matrix(y_test,predicted_classes)